In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from dotenv import load_dotenv
import pandas as pd
import os

load_dotenv("../.env")
user = os.getenv('USERR')
password = os.getenv('PASSW')

In [2]:
op = webdriver.ChromeOptions()
op.add_argument("--headless")

driver = webdriver.Chrome('../chromedriver', options=op)
driver.get('https://www.saes.upiita.ipn.mx/')

# Finding the input tags
user_in = driver.find_element_by_css_selector('#ctl00_leftColumn_LoginUser_UserName')
pas_in = driver.find_element_by_css_selector('.passwordEntry')
captcha_in = driver.find_element_by_css_selector("#ctl00_leftColumn_LoginUser_CaptchaCodeTextBox")

# Screenshot for the captcha
driver.save_screenshot("screen.png")

# Sending the user data to the form
user_in.send_keys(user)
pas_in.send_keys(password)
captcha_in.send_keys(input("Captcha:"))
pas_in.send_keys(Keys.RETURN)

In [3]:
# Going to the ocupation page in the same session
driver.get('https://www.saes.upiita.ipn.mx/Academica/Ocupabilidad_grupos.aspx')

# Selecting actual school period
driver.find_element_by_id("ctl00_mainCopy_rblEsquema_0").click()


# Iterating over the 3 carreers
careers = {"M" : [], "B": [], "T": []}
columns_name = ["Grupo", "Semestre", "Cupo", "Inscritos", "Disponibles", "Asignatura"]
for career in careers.keys():
    # Finding the carreer selector 
    career_finder = driver.find_element_by_id("ctl00_mainCopy_dpdcarrera")
    career_selector = Select(career_finder)
    # Selecting which career to find
    career_selector.select_by_value(career)
    # Getting a single string for all the web table
    ocupability_str = driver.find_element_by_id("ctl00_mainCopy_GrvOcupabilidad").text
    # Split the rows for each \n into a single string
    rows = ocupability_str.split("\n")
    # Separate in columns by the spaces between them
    ocupability_processed = [col.split(" ") for col in rows[1:]]
    # the course name gets separated, let's fix it
    for row in ocupability_processed:
        # slicing the course name
        course_name_slice = row[2:-4]
        # concatenating
        course_str = " ".join(course_name_slice)
        # deleting the separated course name string
        del row[1:-4]
        # appending the concatenated string
        row.append(course_str)
        row.append(course_str+row[0])
        careers[career].append(row)

In [ ]:
careers['M']

In [5]:
columns_name = ["Grupo", "Semestre", "Cupo", "Inscritos", "Disponibles", "Asignatura", "id"]

In [25]:
M_courses = {"courses": [], "id": "M"}
B_courses = {"courses": [], "id": "B"}
T_courses = {"courses": [], "id": "T"}
# for course in careers["M"]:
#     courses["courses"].append({k:v for k,v in zip(columns_name, course)})


In [26]:
courses_api_format = [M_courses, B_courses, T_courses]

In [19]:
courses_api_format

[{'courses': [], 'id': 'M'},
 {'courses': [], 'id': 'B'},
 {'courses': [], 'id': 'T'}]

In [ ]:
list(zip(careers.values(), courses_api_format))

In [22]:
from azure.cosmos import CosmosClient
from azure.cosmos import PartitionKey
from dotenv import load_dotenv
load_dotenv('../.env')
HOST = os.getenv('HOST')
MASTER_KEY = os.getenv('KEY')
DATABASE_ID = os.getenv('DBID')

In [24]:
client = CosmosClient(HOST, {'masterKey': MASTER_KEY})
db = client.get_database_client(DATABASE_ID)
container_prueba = db.get_container_client("Prubea")

In [28]:
for career, for_api in zip(careers.values(), courses_api_format):
    for course in career:
        # print(course)
        for_api["courses"].append({k:v for k,v in zip(columns_name, course)})
        container_prueba.upsert_item(for_api)

In [ ]:
container_prueba.upsert_item(courses)